In [1]:
import tensorflow as tf
import numpy as np
import math
import pandas as pd
from keras.models import Sequential , load_model
from tensorflow.keras.callbacks import TensorBoard
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import os
import time

C:\Users\Ohuan\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ohuan\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ohuan\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ohuan\Anaconda3\envs\tensorflow\lib\site-packages

In [2]:
# Reading in the file
TRAINING_DATA = pd.read_csv('training.csv')
TEST_DATA = pd.read_csv('test.csv')

# Check the row to see if there is a nan data.
def check_for_nan(row):
    for i in range(len(row)):
        if math.isnan(row[i]):
            return True
    return False


TRAIN_IMG = []
TRAIN_POINTS = []
counter = 0
for i in range(len(TRAINING_DATA)):
    ROW_DATA = TRAINING_DATA.iloc[i, :-1]  # Get the ith row of data
    if check_for_nan(ROW_DATA) == False:
        DATA_IMAGE = TRAINING_DATA.iloc[i, -1]  # Get last column of the ith row which should be the image
        DATA_IMAGE = np.array(DATA_IMAGE.split(" ")).astype(int)  # Convert into numpy array of int
        DATA_IMAGE = np.reshape(DATA_IMAGE, (96, 96))
        DATA_IMAGE = DATA_IMAGE / 255  # Normalize between 0.0 - 1.0

        TRAIN_IMG.append(DATA_IMAGE)

        X_Y_POINTS = TRAINING_DATA.iloc[i, :-1].astype(int).values
        X_Y_POINTS = X_Y_POINTS / 96  # Normalize the points

        TRAIN_POINTS.append(X_Y_POINTS)

TRAIN_IMG = np.array(TRAIN_IMG)
TRAIN_IMG = np.reshape(TRAIN_IMG, (TRAIN_IMG.shape[0], TRAIN_IMG.shape[1], TRAIN_IMG.shape[2], 1))
TRAIN_POINTS = np.array(TRAIN_POINTS)

TEST_IMG = []
for i in range(len(TEST_DATA)):
    DATA_IMAGE = TEST_DATA.iloc[i, -1]
    DATA_IMAGE = np.array(DATA_IMAGE.split(' ')).astype(int)
    DATA_IMAGE = np.reshape(DATA_IMAGE, (96, 96))
    DATA_IMAGE = DATA_IMAGE / 255

    TEST_IMG.append(DATA_IMAGE)

TEST_IMG = np.array(TEST_IMG)

In [3]:
# This method test different models with different number of dense layers and convolutional layers. We also test different layers sizes
# If you want to look at tensorboard go to location of the log folder and run in terminal:
# tensorboard --logdir=logs/
# Customize to your needes

def test_diff_model():
    
    DENSE_LAYERS = [0, 1, 2]
    LAYER_SIZES = [32, 64, 128]
    CONV_LAYERS = [1, 2, 3]

    for dense_layer in DENSE_LAYERS:
        for layer_size in LAYER_SIZES:
            for conv_layer in CONV_LAYERS:
                NAME = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
                log_dir = os.path.join(
                    "logs",
                    NAME,
                )
                tensorboard = TensorBoard(log_dir)
    
                model = Sequential()
                model.add(Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=(96, 96, 1), activation='relu'))
                model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

                for l in range(conv_layer- 1):
                    model.add(Conv2D(layer_size, kernel_size=3, strides=2, padding='same', activation='relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

                model.add(Flatten())

                for _ in range(dense_layer):
                    model.add(Dense(layer_size, activation='relu'))
                    model.add(Dropout(0.1))

                model.add(Dense(30))
                               
                model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
                
                model.fit(TRAIN_IMG, TRAIN_POINTS, epochs=300, batch_size=200,callbacks = [tensorboard])
               

In [4]:
# Uncomment to run the method above when needed

#test_diff_model()

In [5]:
#Edit to number of layers that you want found best to fit

NUM_CONV_LAYER = 1
NUM_DENSE_LAYER = 0


def get_model():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=(96, 96, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    
    for l in range(NUM_CONV_LAYER- 1):
        model.add(Conv2D(layer_size, kernel_size=3, strides=2, padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    
    model.add(Flatten())
    
    for l in range(NUM_DENSE_LAYER):
        model.add(Dense(layer_size, activation='relu'))
        model.add(Dropout(0.1))
    
    
   
    model.add(Dense(30))
    return model


def compile_model(model): # Compile the model
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])

def train_model(model): # Fit the model
    model.fit(TRAIN_IMG, TRAIN_POINTS, epochs=300, batch_size=200)

def save_model(model, saveFileName): # Save the model
    model.save(saveFileName + '.h5')

def load_trained_model(saveFileName): # Load the model
    return load_model(saveFileName + '.h5')

In [6]:
# Uncomment when you need to run and save model

"""
model = get_model()
compile_model(model)
train_model(model)
save_model(model,"themodel")
"""

'\nmodel = get_model()\ncompile_model(model)\ntrain_model(model)\nsave_model(model,"themodel")\n'

In [7]:
import cv2 as cv

In [10]:
model = load_trained_model("themodel") # Loading the model

face_cascade = cv.CascadeClassifier('cascades/haarcascade_frontalface_default.xml') # Haarcasecade which we use to detect our face

cap = cv.VideoCapture(0)

counter = 0 # Counter for changing filters

while (cap.isOpened()):
    
    ret, frame = cap.read() # Capturing image
    
    if (ret):

        # Chaning frame to greyscale
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
        # Using the face_cascade
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        
        for (x, y, w, h) in faces:
            
            test_frame = gray[y:y + h, x:x + w]
            width_original = test_frame.shape[1]  # Width of region where face is detected
            height_original = test_frame.shape[0] # Height of region where face is detected
            test_frame = cv.resize(test_frame, (96, 96))
            test_frame = test_frame / 255
            test_frame = np.reshape(test_frame, (1, 96, 96, 1))
            
            COORDINATES = model.predict(test_frame)[0] # Getting the coordinates using our model for our face

        
            X_COORDINATES = COORDINATES[0::2] # Getting every x coordinate 
            Y_COORDINATES = COORDINATES[1::2] # Getting every y coordinate

            DENORM_X_COORD = X_COORDINATES * width_original # Denormalizing the coordinates since we normalize image
            DENORM_Y_COORD = Y_COORDINATES * height_original
            
            # Making a frame for coordinates and filter
            COORDINATE_FRAME = frame[y:y + h, x:x + w]
            FILTER_FRAME = np.copy(frame) 
            
            # Plot all coordinate on a image 
            for i in range(len(X_COORDINATES)):
                cv.circle(COORDINATE_FRAME, (DENORM_X_COORD[i], DENORM_Y_COORD[i]), 2, (255, 255, 0), -1)
           
            RIGHT_OUTER_EYEBROW = (int(DENORM_X_COORD[9]), int(DENORM_Y_COORD[9])) 
            LEFT_EYE_COORDINATES = (int(DENORM_X_COORD[3]), int(DENORM_Y_COORD[3]))
            RIGHT_EYE_COORDINATES = (int(DENORM_X_COORD[5]), int(DENORM_Y_COORD[5]))
            

            GLASSES_WIDTH = RIGHT_EYE_COORDINATES[0] - LEFT_EYE_COORDINATES[0] # The width of the glasses which we are gonna resize. Going to be distance from coor of right eye to left eye

            FILTER_FRAME = cv.cvtColor(FILTER_FRAME, cv.COLOR_BGR2BGRA) # Using alpha channel to combine filter images
            
            # Counters to change our glasses filter we are going to use
            if counter >=0 and counter < 20:
                glasses = cv.imread('filters/glasses.png', -1)
                counter +=1
            elif counter >=20 and counter <40:
                glasses = cv.imread('filters/darkglasses.png', -1)
                counter+=1
            else:
                glasses = cv.imread('filters/sportsunglasses.png', -1)
                counter +=1
                if counter == 60:
                    counter=0
                
            glasses = cv.resize(glasses, (abs(GLASSES_WIDTH)*2 , 150)) # Resizeing glasses 
            glasses_width, glasses_height, notuse = glasses.shape
            
            # Double for loop for replacing pixels with glasses.
            # Coordinates from right_eyebrow_outer is now used in our main frame which is larger than just in face_cascade frame, so we have to do some tweaking
            
            for j in range(0, glasses_height):
                for i in range(0, glasses_width):
                    if glasses[i, j][3] != 0: # Making sure we are only copying pixels of glasses
                        FILTER_FRAME[RIGHT_OUTER_EYEBROW[0] + i + x - 80 , RIGHT_OUTER_EYEBROW[1] + j + y] = glasses[i, j]
                              
            FILTER_FRAME = cv.cvtColor(FILTER_FRAME, cv.COLOR_BGRA2BGR)    
            
            #Showing the different frames
            cv.imshow("Cam With Filter",FILTER_FRAME)
            cv.imshow('Cam with coordinates', COORDINATE_FRAME)
        cv.imshow('Cam', frame)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv.destroyAllWindows()